In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
train=pd.read_csv('/content/drive/My Drive/data/open_data/train.csv', index_col=0)
test=pd.read_csv('/content/drive/My Drive/data/open_data/test_x.csv', index_col=0)
submission=pd.read_csv('/content/drive/My Drive/data/open_data/sample_submission.csv', index_col=0)  



In [ ]:
X = train.drop('voted', axis = 1)
y = train['voted']

In [ ]:
print(X.race.value_counts())


In [ ]:
print(X.age_group.value_counts())


In [ ]:
print(X.gender.value_counts())


In [ ]:
print(X.religion.value_counts())


In [ ]:
print("원본 데이터 칼럼 : ", list(X.columns), "\n")
X_dummies = pd.get_dummies(X)
print("get_dummies된 데이터 칼럼 : ", list(X_dummies.columns))

In [ ]:
print("X: {}\tX_dummies: {}".format(X.shape, X_dummies.shape))


In [ ]:
X = pd.get_dummies(X)

In [ ]:
test = pd.get_dummies(test)
test.shape

In [ ]:
# 칼럼 개수 변화
print("X : {}\ntest : {}".format(X.shape, test.shape))
# 인코딩 확인
print("Encoding Success") if list(X.columns) == list(test.columns) else list(test.columns)


In [ ]:
# nan 값 메꾸기
X = X.fillna(X.mean())
# 중복 값 제거
X.drop_duplicates(keep='first', inplace = True)
# 비교 -> nan 없음
X.shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
# 데이터 스케일링 -> 민맥스/스텐다드 모두 성능 비슷함
scaler=MinMaxScaler()
scaler.fit(X)
X=scaler.transform(X)
# 테스트 데이터도 동일 스케일러로
test=scaler.transform(test)

In [ ]:
!pip install bayesian-optimization

In [ ]:
import lightgbm as lgbm
from bayes_opt import BayesianOptimization
from sklearn.metrics import roc_auc_score, make_scorer
from sklearn.model_selection import cross_validate

In [ ]:
#목적함수 생성
def lgbm_cv(learning_rate, num_leaves, max_depth, min_child_weight, colsample_bytree, feature_fraction, bagging_fraction, lambda_l1, lambda_l2):
    model = lgbm.LGBMClassifier(learning_rate=learning_rate,
                                n_estimators = 300,
                                #boosting = 'dart',
                                num_leaves = int(round(num_leaves)),
                                max_depth = int(round(max_depth)),
                                min_child_weight = int(round(min_child_weight)),
                                colsample_bytree = colsample_bytree,
                                feature_fraction = max(min(feature_fraction, 1), 0),
                                bagging_fraction = max(min(bagging_fraction, 1), 0),
                                lambda_l1 = max(lambda_l1, 0),
                                lambda_l2 = max(lambda_l2, 0)
                               )
    scoring = {'roc_auc_score': make_scorer(roc_auc_score)}
    result = cross_validate(model, X, y, cv=5, scoring=scoring)
    auc_score = result["test_roc_auc_score"].mean()
    return auc_score

In [ ]:
# 입력값의 탐색 대상 구간
pbounds = {'learning_rate' : (0.0001, 0.05),
           'num_leaves': (300, 600),
           'max_depth': (2, 25),
           'min_child_weight': (30, 100),
           'colsample_bytree': (0, 0.99),
           'feature_fraction': (0.90, 0.99),
           'bagging_fraction': (0.70, 0.99),
           'lambda_l1' : (0, 0.99),
           'lambda_l2' : (0, 0.99),
          }

In [ ]:
lgbmBO = BayesianOptimization(f = lgbm_cv, pbounds = pbounds, verbose = 2, random_state = 0 )


In [ ]:
lgbmBO.maximize(init_points=5, n_iter = 20, acq='ei', xi=0.01)


In [ ]:
lgbmBO.max

In [ ]:
#파라미터 적용
fit_lgbm = lgbm.LGBMClassifier(learning_rate=lgbmBO.max['params']['learning_rate'],
                               num_leaves = int(round(lgbmBO.max['params']['num_leaves'])),
                               max_depth = int(round(lgbmBO.max['params']['max_depth'])),
                               min_child_weight = int(round(lgbmBO.max['params']['min_child_weight'])),
                               colsample_bytree=lgbmBO.max['params']['colsample_bytree'],
                               feature_fraction = max(min(lgbmBO.max['params']['feature_fraction'], 1), 0),
                               bagging_fraction = max(min(lgbmBO.max['params']['bagging_fraction'], 1), 0),
                               lambda_l1 = lgbmBO.max['params']['lambda_l1'],
                               lambda_l2 = lgbmBO.max['params']['lambda_l2']
                               )

In [ ]:
model = fit_lgbm.fit(X,y)


In [ ]:
import joblib
joblib.dump(model, 'lgbmBO_201006.pkl')

In [ ]:
pred_y = model.predict(test)


In [ ]:
submission['voted']=pred_y
submission.to_csv('lgbmBO_201006.csv')